In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import re
from pathlib import Path


def first_file_meta(var):
    f = sorted(base.glob(f"cmems_{var}_daily_*.nc"))[0]
    m = re.match(rf"cmems_{var}_daily_(\d{{4}})_(.+)_(\d+(?:\.\d+)?deg)\.nc", f.name)
    area = m.group(2)
    res = m.group(3)
    return area, res

def open_unified(var):
    files = sorted(base.glob(f"cmems_{var}_daily_*.nc"))
    ds = xr.open_mfdataset(files, combine="by_coords")
    return ds

def drop_feb29_index(time_index):
    is_feb29 = (time_index.month == 2) & (time_index.day == 29)
    return time_index[~is_feb29]

def add_doy_noleap(ds):
    t = pd.DatetimeIndex(ds.time.values)
    mask = (t.month == 2) & (t.day == 29)
    t2 = t.copy()
    t2 = t2[~mask]
    ds2 = ds.sel(time=~mask)
    doy = pd.Index(t2.dayofyear)
    doy = doy.where(doy < 60, doy - 1)
    ds2 = ds2.assign_coords(doy=("time", doy.values))
    return ds2

def weekly_from_jan1(ds):
    t = pd.DatetimeIndex(ds.time.values)
    df = pd.DataFrame({"time": t, "year": t.year, "doy": t.dayofyear})
    mask = (df["time"].dt.month == 2) & (df["time"].dt.day == 29)
    df = df.loc[~mask].copy()
    df["doy_noleap"] = np.where(df["doy"] < 60, df["doy"], df["doy"] - 1)
    df["week_idx"] = ((df["doy_noleap"] - 1) // 7) + 1
    df["week_label"] = df["year"].astype(str) + "-" + df["week_idx"].astype(str).str.zfill(2)
    lab = xr.DataArray(df["week_label"].values, coords={"time": ds.time.values[~mask.values]}, dims=("time",))
    ds2 = ds.sel(time=~mask.values)
    grp = ds2.groupby(lab)
    wmean = grp.mean("time", skipna=True)
    wtime = grp.map(lambda x: x.time.max())
    wmean = wmean.assign_coords(time=("week_label", wtime.values))
    wmean = wmean.swap_dims({"week_label": "time"}).sortby("time")
    wmean = wmean.assign_coords(week_in_year=((wmean["time"].dt.dayofyear - 1) // 7) + 1, year=wmean["time"].dt.year)
    return wmean

def daily_climatology(ds):
    ds_nl = add_doy_noleap(ds)
    clim = ds_nl.groupby("doy").mean("time", skipna=True)
    return clim

def daily_anomalies(ds):
    ds_nl = add_doy_noleap(ds)
    clim = ds_nl.groupby("doy").mean("time", skipna=True)
    anom = ds_nl.groupby("doy") - clim
    return anom

def weekly_climatology(ds_weekly):
    clim = ds_weekly.groupby("week_in_year").mean("time", skipna=True)
    return clim

def weekly_anomalies(ds_weekly):
    clim = weekly_climatology(ds_weekly)
    anom = ds_weekly.groupby("week_in_year") - clim
    return anom

def period_from_time(ds):
    t = pd.DatetimeIndex(ds.time.values)
    return f"{t.min().year}-{t.max().year}"



In [2]:

base = Path("/home/jupyter-daniela/suyana/sources/biogeochem/mercator")
outpath = Path("/home/jupyter-daniela/suyana/peru_production/features/")
dataset_name = "cmems"

for var in ["chl", "o2"]:
    area, res = first_file_meta(var)
    ds = open_unified(var)
    per = period_from_time(ds)
    (outpath / f"{dataset_name}_{var}_daily_{per}_{area}_{res}.nc").write_bytes(b"")
    ds.to_netcdf(outpath / f"{dataset_name}_{var}_daily_{per}_{area}_{res}.nc")
    ds_daily_anom = daily_anomalies(ds)
    ds_daily_anom.to_netcdf(outpath / f"{dataset_name}_{var}_daily-anom_{per}_{area}_{res}.nc")
    ds_weekly = weekly_from_jan1(ds)
    ds_weekly.to_netcdf(outpath / f"{dataset_name}_{var}_weekly_{per}_{area}_{res}.nc")
    ds_weekly_anom = weekly_anomalies(ds_weekly)
    ds_weekly_anom.to_netcdf(outpath / f"{dataset_name}_{var}_weekly-anom_{per}_{area}_{res}.nc")

/tmp/ipykernel_2553599/107238013.py:17: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'depth' ('depth',) The recommendation is to set join explicitly for this case.
  ds = xr.open_mfdataset(files, combine="by_coords")


: 